In [1]:
pip install torch torchvision


Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader


In [9]:
# Data transformation with normalization
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


# Download and load the training data
trainset = datasets.CIFAR10('~/.pytorch/CIFAR_data/', download=True, train=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.CIFAR10('~/.pytorch/CIFAR_data/', download=True, train=False, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=True)


Files already downloaded and verified
Files already downloaded and verified


In [10]:
from torch import nn, optim
import torch.nn.functional as F

class CIFARClassifier(nn.Module):
    def __init__(self):
        super(CIFARClassifier, self).__init__()
        self.fc1 = nn.Linear(3 * 32 * 32, 512)  # CIFAR images are 3x32x32
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)  # 10 classes in CIFAR-10

    def forward(self, x):
        x = x.view(x.shape[0], -1)  # Flatten the images
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [11]:
model = CIFARClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

epochs = 10
for epoch in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        print(f"Epoch {epoch+1} - Training loss: {running_loss/len(trainloader)}")


Epoch 1 - Training loss: 1.6919321922390052
Epoch 2 - Training loss: 1.4347708383027244
Epoch 3 - Training loss: 1.3179574992955494
Epoch 4 - Training loss: 1.2252861847505545
Epoch 5 - Training loss: 1.14581098954391
Epoch 6 - Training loss: 1.0751355497733406
Epoch 7 - Training loss: 1.0081267295133731
Epoch 8 - Training loss: 0.9396503904591436
Epoch 9 - Training loss: 0.8748232191404723
Epoch 10 - Training loss: 0.8116645702087056


In [12]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the 10000 test images: {100 * correct / total}%')


Accuracy of the model on the 10000 test images: 54.36%
